# dbachecks PR check on Beard NUC - MultipleInstances

Rob uses this process to make sure that changes will work in an AD estate with a variety of versions

First we must check the status of the local source control and ensure that we are in the PR branch

In [1]:
cd D:\OneDrive\Documents\GitHub\dbachecks
git status

On branch development
Your branch is ahead of 'origin/development' by 13 commits.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   checks/Instance.Tests.ps1

no changes added to commit (use "git add" and/or "git commit -a")


First we need to import the module from source control - I normally use verbose to see how it is going


In [2]:
Import-Module .\dbachecks.psd1
$null= Reset-DbcConfig

Excellent, that worked. Now to run some checks against a single instance  

I don't care if the test pass or fail, I am only interested in any 

> Describe block has Error

or

> Error in Script

type failures

Multiple Instances at parameter

In [12]:
$Sqlinstances = (Get-VM -ComputerName BEARDNUC, BEARDNUC2 -Name *SQL* | Where State -eq 'Running').Name

Invoke-DbcCheck -SqlInstance $Sqlinstances -Check LoginCheckPolicy, LoginPasswordExpiration,LoginMustChange, SaExist, BuiltInAdmin

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Instance.Tests.ps1' with Tags LoginCheckPolicy', 'LoginPasswordExpiration', 'LoginMustChange', 'SaExist', 'BuiltInAdmin

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Instance.Tests.ps1

  Describing Login SA cannot exist

    Context Checking that a login named sa does not exist on SQL2005Ser2003
      [+] sa login does not exist on SQL2005Ser2003 1ms

  Describing Login BUILTIN Administrators cannot exist

    Context Checking that a login named BUILTIN\Administrators does not exist on SQL2005Ser2003
      [-] BUILTIN\Administrators login does not exist on SQL2005Ser2003 4ms
        Expected 0, because We expected no login to exist with the name BUILTIN\Administrators, but got 1.
        1011:     $AllInstanceInfo.BuiltInAdmin.Exist | Sh

Set the config and run again

In [11]:
Get-DbaServerRole -SQLInstance sql2019n5 -ServerRole "sysadmin"


ComputerName : SQL2019N5
InstanceName : MSSQLSERVER
SqlInstance  : SQL2019N5
Role         : sysadmin
Login        : {sa, THEBEARD\Rob, NT SERVICE\SQLWriter, NT SERVICE\Winmgmt…}
Owner        : sa
IsFixedRole  : True
DateCreated  : 13/04/2009 12:59:06
DateModified : 13/04/2009 12:59:06




In [9]:
@(Get-DbaLogin -SQLInstance sql2019n5 -Type SQL | Where-Object { $_.PasswordPolicyEnforced -eq $false -and $_.IsDisabled -eq $false}).Count

1


In [10]:
@(Get-DbaLogin -SqlInstance sql2019n5 -Login sa).Count

1
